In [2]:
setwd("/home/pasoneto/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")
cat(howto_data)

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



How to use datasets
 
call    base()             for real values
call    z_scored()         for normalized
call    min_maxed()        for normalized2
call    upsampled_album()  for binded upsamplped albums
call    list_upsampled()   for list of upsampled albums
call    low_z()            for normalized low level
call    low_raw()          for raw  low level

# Writing data for lstm

In [ ]:
oi = upsampled_album()

ltreino = (nrow(oi)/16)*0.8
lteste = (nrow(oi)/16)*0.2

colnames(oi) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")

treino = oi[1:(ltreino*16)]
teste = oi[((ltreino*16)+1):nrow(oi)]

(nrow(treino)+nrow(teste))==nrow(oi)

write.csv(treino, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/lstm_treino_teste/treino.csv")
write.csv(teste, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/lstm_treino_teste/teste.csv")

# Writing data for dissimilarity

In [ ]:
library("tidyverse")

#Import list of dissimilarities
oi = list_upsampled() #upsampled using bilinear interpolation

# #Renaming columns and resetting track_number
for(i in 1:length(oi)){
    oi[[i]]$`0` <- seq(1, 16, 1)
    colnames(oi[[i]]) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")
}

#Generating dissimilarities
albums = dissim_gen(oi)

#Binding albums
albums = do.call(rbind, albums)

#Remaking track numbers
albums = data.frame(albums)
albums$track_number = rep(seq(1, 16, 1), length(oi) )

#Removing NAs
ha <- split(albums, seq(nrow(albums)))
for(i in 1:length(ha)){
    ha[[i]] <- as.list(ha[[i]])
}

for(i in 1:length(ha)){
    ha[[i]] = unlist(ha[[i]], use.names = FALSE)[!is.na(unlist(ha[[i]], use.names = FALSE))]
}
#####Removing tracks with too many NAs (due to upsampling)
final = c()
for(i in 1:length(ha)){
    if(length(ha[[i]]) == 16){
        final[[i]] = ha[[i]]
    }
}

final <- rlist::list.rbind(final)
final <- data.frame(final)
colnames(final) <- c("x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11", "x12", "x13", "x14", "x15", "track_number")

final$position = "middle"
for(i in 1:NROW(final)){
       if(final$track_number[[i]] == 1){
           final$position[[i]] = "edge"
       }
       if(final$track_number[[i]] == 16){
            final$position[[i]] = "edge"
       }
}

In [ ]:
edge = final %>% filter(position == "edge")
middle = final %>% filter(position == "middle") %>% dplyr::sample_n(8890)

overall = bind_rows(middle, edge)

## 75% of the sample size
smp_size <- floor(0.75 * nrow(overall))
train_ind <- sample(seq_len(nrow(overall)), size = smp_size)

## Separating
train <- overall[train_ind, ]
train %<>% select(!track_number)
test <- overall[-train_ind, ]
test %<>% select(!track_number)

In [ ]:
#Train test split
write.csv(train, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/classificacao_edges/train.csv")
write.csv(test, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/classificacao_edges/test.csv")

In [ ]:
#Writting for later usage
write.csv(final, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/classificacao_edges/classificacao_edges.csv")

# Writting dissimilarities 2

In [ ]:
dt = z_scored()

In [ ]:
dissim_by_track <- function(dt, colname) {
    colname <- enquo(colname) 
    dt %>% 
        select(!!colname) %>%
        as.matrix() %>% 
        cluster::daisy() %>%
        as.matrix() %>% 
        colMeans() -> result 
    return(result)
}

edge_detector =
    function(d){
        for(i in 1:NROW(d)){
               if(d$track_number[[i]] == 1){
                   d$position[[i]] = "edge"
               }
               if(d$track_number[[i]] == NROW(d)){
                    d$position[[i]] = "edge"
               }
        }
        return(d$position)
    }

In [ ]:
dt$position = "middle"
dt = split(dt, dt$album_id)

In [ ]:
for(i in 1:length(dt)){
    
    dt[[i]]$valence = dissim_by_track(dt[[i]], valence)
    dt[[i]]$energy = dissim_by_track(dt[[i]], energy)
    dt[[i]]$loudness = dissim_by_track(dt[[i]], loudness)
    dt[[i]]$tempo = dissim_by_track(dt[[i]], tempo)
    
    dt[[i]]$position = edge_detector(dt[[i]])
    
}

In [ ]:
dt = dplyr::bind_rows(dt)

In [ ]:
# ## 75% of the sample size
smp_size <- floor(0.75 * nrow(dt))
train_ind <- sample(seq_len(nrow(dt)), size = smp_size)

# ## Separating
train <- dt[train_ind, ]
train %<>% select(!track_number)
test <- dt[-train_ind, ]
test %<>% select(!track_number)

In [ ]:
#Train test split
write.csv(train, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/classificacao_edges/train.csv")
write.csv(test, "/home/pasoneto/Documents/github/doc_suomi/data/treated_data/classificacao_edges/test.csv")

# Writting data for low

In [6]:
oi = low_minmax()

# Removing albums with small tracks
oi %<>% 
    group_by(track_id) %>%
    mutate(nsamples = NROW(valence)) %>%
    filter(nsamples < 5)
a_out <- unique(oi$album_id)
data = low_minmax()

data %<>%
    filter(!album_id %in% a_out) %>% 
    ungroup() %>%
    group_by(track_id) %>%
    mutate(track_section = seq(1, NROW(track_id), 1))

In [7]:
data %<>%
    filter(!album_id %in% a_out) %>% 
    ungroup() %>%
    group_by(track_id) %>%
    mutate(track_section = seq(1, NROW(track_id), 1)) %>%
    mutate(track_section = lsr::quantileCut(track_section, 5, labels = c("a", "b", "c", "d", "e"))) %>%
    ungroup() %>% group_by(album_id, track_id, track_section) %>%
    summarise(danceability = mean(danceability),
              energy = mean(energy), 
              loudness_overall = mean(loudness_overall),
              speechiness = mean(speechiness),
              acousticness = mean(acousticness),
              instrumentalness = mean(instrumentalness), 
              liveness = mean(liveness),
              valence = mean(valence),
              tempo_overall = mean(tempo_overall), 
              duration_ms = mean(duration_ms),
              time_signature = mean(time_signature),
              track_number = mean(track_number), 
              loudness_continuous = mean(loudness_continuous),
              tempo_continuous = mean(tempo_continuous), 
              tempo_confidence = mean(tempo_confidence), 
              key_confidence = mean(key_confidence),
              mode_confidence = mean(mode_confidence), 
              time_signature_confidence = mean(time_signature_confidence)) %>%
    ungroup() %>%
    arrange(album_id) %>% select(!track_section)

`summarise()` regrouping output by 'album_id', 'track_id' (override with `.groups` argument)



In [8]:
# Cumulative
data %>%
    group_by(album_id, track_id) %>%
    summarise(danceability_cum = mean(danceability),
           energy_cum = mean(energy),
           loudness_overall_cum = mean(loudness_overall),
           speechiness_cum = mean(speechiness),
           acousticness_cum = mean(acousticness),
           instrumentalness_cum = mean(instrumentalness), 
           liveness_cum = mean(liveness),
           valence_cum = mean(valence),
           tempo_overall_cum = mean(tempo_overall), 
           duration_ms_cum = mean(duration_ms),
           time_signature_cum = mean(time_signature),
           loudness_continuous_cum = mean(loudness_continuous),
           tempo_continuous_cum = mean(tempo_continuous),
           tempo_confidence_cum = mean(tempo_confidence),
           key_confidence_cum = mean(key_confidence),
           mode_confidence_cum = mean(mode_confidence),
           time_signature_confidence_cum = mean(time_signature_confidence)) %>%
    ungroup() %>% group_by(album_id) %>%
    mutate(danceability_cum = cumsum(danceability_cum),
           energy_cum = cumsum(energy_cum),
           loudness_overall_cum = cumsum(loudness_overall_cum),
           speechiness_cum = cumsum(speechiness_cum),
           acousticness_cum = cumsum(acousticness_cum),
           instrumentalness_cum = cumsum(instrumentalness_cum), 
           liveness_cum = cumsum(liveness_cum),
           valence_cum = cumsum(valence_cum),
           tempo_overall_cum = cumsum(tempo_overall_cum), 
           duration_ms_cum = cumsum(duration_ms_cum),
           time_signature_cum = cumsum(time_signature_cum),
           loudness_continuous_cum = cumsum(loudness_continuous_cum),
           tempo_continuous_cum = cumsum(tempo_continuous_cum),
           tempo_confidence_cum = cumsum(tempo_confidence_cum),
           key_confidence_cum = cumsum(key_confidence_cum),
           mode_confidence_cum = cumsum(mode_confidence_cum),
           time_signature_confidence_cum = cumsum(time_signature_confidence_cum)) -> cummed

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



In [9]:
oi = merge(x = cummed, y = data, by.x = c("album_id", "track_id"), by.y = c("album_id", "track_id"))

In [10]:
greater = function(x){
    final = c('start')
    for(i in 1:(length(x)-1)){ 
        if(x[i+1] > x[i]){
            final = c(final, 'greater')            
        } else{
            final = c(final, 'smaller')
        }
    }
    return(final)
}

oi %<>% 
    ungroup() %>%
    mutate(danceability_cum = minmax(danceability_cum, 1, 2),
           energy_cum = minmax(energy_cum, 1, 2),
           loudness_overall_cum = minmax(loudness_overall_cum, 1, 2),
           speechiness_cum = minmax(speechiness_cum, 1, 2),
           acousticness_cum = minmax(acousticness_cum, 1, 2),
           instrumentalness_cum = minmax(instrumentalness_cum, 1, 2), 
           liveness_cum = minmax(liveness_cum, 1, 2),
           valence_cum = minmax(valence_cum, 1, 2),
           tempo_overall_cum = minmax(tempo_overall_cum, 1, 2), 
           duration_ms_cum = minmax(duration_ms_cum, 1, 2),
           time_signature_cum = minmax(time_signature_cum, 1, 2),
           loudness_continuous_cum = minmax(loudness_continuous_cum, 1, 2),
           tempo_continuous_cum = minmax(tempo_continuous_cum, 1, 2),
           tempo_confidence_cum = minmax(tempo_confidence_cum, 1, 2),
           key_confidence_cum = minmax(key_confidence_cum, 1, 2),
           mode_confidence_cum = minmax(mode_confidence_cum, 1, 2),
           time_signature_confidence_cum = minmax(time_signature_confidence_cum, 1, 2))

In [12]:
oi %>% group_by(album_id) %>%
    mutate(valence_cat  = greater(valence),
           energy_cat   = greater(energy),
           loudness_cat = greater(loudness_overall),
           tempo_cat    = greater(tempo_overall)) %>%
    ungroup() %>% 
    group_by(album_id, track_number) %>%
    mutate(valence_cat = valence_cat[1],
           energy_cat = energy_cat[1],
           loudness_cat = loudness_cat[1],
           tempo_cat = tempo_cat[1]) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/lstm/lstm.csv")